In [120]:
# Load dependence
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import warnings
from sklearn.exceptions import ConvergenceWarning
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Set number of out-of-sample test and fold
num_oos = 9
num_fold = 10

In [54]:
#Load raw data
df = pd.read_csv('./RuCHFunctionalizationDataset/dataset.csv')
df_DG = pd.read_csv('./RuCHFunctionalizationDataset/DG.csv')
df_RX = pd.read_csv('./RuCHFunctionalizationDataset/RX.csv')

DG_smiles = df['DG'].to_list()
RX_smiles = df['RX'].to_list()
cat_smiles = df['catalyst'].to_list()
sol_smiles = df['solvent'].to_list()
lig_smiles = df['ligand'].to_list()
ad_smiles = df['addictive'].to_list()

target = df['tag'].to_numpy()
target_DG_columns = ['lumo', 'EA', 'bv-3.5', ]
target_DG = df_DG[target_DG_columns].to_numpy()
target_RX_columns = ['somo', 'bv-3.5', 'spin']
target_RX = df_RX[target_RX_columns].to_numpy()

In [56]:
# Generate MOL files from SMILES
def gen_mol_form_smi(smi_list):
    mol_list = []
    for smi in smi_list:
        if isinstance(smi, str) == True :
            mol = Chem.MolFromSmiles(smi)
            if mol == None:
                print(smi)
            mol_list.append(mol)
        else:
            mol = 0
            mol_list.append(mol)
    return mol_list

DG_mols =  gen_mol_form_smi(DG_smiles)
RX_mols =  gen_mol_form_smi(RX_smiles)
cat_mols =  gen_mol_form_smi(cat_smiles)
sol_mols =  gen_mol_form_smi(sol_smiles)
lig_mols =  gen_mol_form_smi(lig_smiles)
ad_mols = gen_mol_form_smi(ad_smiles)

In [60]:
# Generate RDKit descriptors from MOL files
descs = [desc_name[0] for desc_name in Descriptors._descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(descs)

def gen_desc_from_mol(mol_list):
    desc_list = []
    for mol in mol_list:
        if mol != 0:
            desc = desc_calc.CalcDescriptors(mol)
            desc_list.append(desc)
        else:
            n = 208
            decs = [0]*n
            desc_list.append(decs)
    decs = np.array(desc_list)
    return decs

DG_descs = gen_desc_from_mol(DG_mols)
RX_descs = gen_desc_from_mol(RX_mols)
cat_descs = gen_desc_from_mol(cat_mols)
sol_descs = gen_desc_from_mol(sol_mols)
lig_descs = gen_desc_from_mol(lig_mols)
ad_descs = gen_desc_from_mol(ad_mols)

In [62]:
# Concatenate mechansitic descriptors with RDKit descriptors
f_p_2 = df_DG['f+2'].to_numpy()
f_p_3 = df_DG['f+3'].to_numpy()
f_p_4 = df_DG['f+4'].to_numpy()
f_p_5 = df_DG['f+5'].to_numpy()
f_p_6 = df_DG['f+6'].to_numpy()

f_m_2 = df_DG['f-2'].to_numpy()
f_m_3 = df_DG['f-3'].to_numpy()
f_m_4 = df_DG['f-4'].to_numpy()
f_m_5 = df_DG['f-5'].to_numpy()
f_m_6 = df_DG['f-6'].to_numpy()

f_0_2 = df_DG['f0-2'].to_numpy()
f_0_3 = df_DG['f0-3'].to_numpy()
f_0_4 = df_DG['f0-4'].to_numpy()
f_0_5 = df_DG['f0-5'].to_numpy()
f_0_6 = df_DG['f0-6'].to_numpy()

Q_2 = df_DG['Q2'].to_numpy()
Q_3 = df_DG['Q3'].to_numpy()
Q_4 = df_DG['Q4'].to_numpy()
Q_5 = df_DG['Q5'].to_numpy()
Q_6 = df_DG['Q6'].to_numpy()

DG_phys = np.column_stack([f_p_2, f_p_3, f_p_4, f_p_5, f_p_6, f_m_2, f_m_3, f_m_4, f_m_5, f_m_6,
                          f_0_2, f_0_3, f_0_4, f_0_5, f_0_6, Q_2, Q_3, Q_4, Q_5, Q_6])

Qc_R = df_RX['Qc'].to_numpy()
f0 = df_RX['f0'].to_numpy()
f_m = df_RX['f-'].to_numpy()
f_p = df_RX['f+'].to_numpy()

RX_phys = np.column_stack([Qc_R, f0, f_m, f_p])

all_descriptors = np.concatenate([DG_descs, RX_descs, cat_descs, lig_descs, sol_descs, ad_descs, DG_phys, RX_phys],axis=1)
all_descriptors = pd.DataFrame(all_descriptors)
all_descriptors = all_descriptors.dropna(axis=1,how='any')
all_descriptors= np.array(all_descriptors)
all_descriptors = np.unique(all_descriptors, axis=1)

DG_descs = pd.DataFrame(DG_descs)
DG_descs = DG_descs.dropna(axis=1,how='any')
DG_descs = np.array(DG_descs)
DG_descs = np.unique(DG_descs, axis=1)

RX_descs = pd.DataFrame(RX_descs)
RX_descs = RX_descs.dropna(axis=1,how='any')
RX_descs = np.array(RX_descs)
RX_descs = np.unique(RX_descs, axis=1)

In [64]:
# Find substituted DG
def ortho_substituted(mol):
    mol_no_H = AllChem.RemoveHs(mol)
    for idx, atom in enumerate(mol_no_H.GetAtoms()):
        if idx == 1 :
            if atom.GetDegree() > 2:
                return True
        elif idx == 5:
            if atom.GetDegree() > 2:
                return True
            
def meta_substituted(mol):
    mol_no_H = AllChem.RemoveHs(mol)
    for idx, atom in enumerate(mol_no_H.GetAtoms()):
        if idx == 2 :
            if atom.GetDegree() > 2:
                return True
        elif idx == 4:
            if atom.GetDegree() > 2:
                return True
            

ortho_sub_list = []
meta_sub_list = []

for idx, mol in enumerate(DG_mols):
    if ortho_substituted(mol) == True:
        ortho_sub_list.append(idx)

for idx, mol in enumerate(DG_mols):
    if meta_substituted(mol) == True:
        meta_sub_list.append(idx)

In [126]:
# Prepare models and evaluation function

def evaluate_classification_accuracy(label, pred, idx, o_list, m_list):
    correct = 0
    #use_correct = 0
    total = len(label)
    for i in range(total):
        if idx[i] in o_list:
            if label[i] == pred[i]:
                correct += 1               
        elif idx[i] in m_list:
            if label[i] == pred[i]:
                correct += 1
        else:
            if label[i] == 0 and pred[i] == 0:
                correct += 1
            elif label[i] == 0 and pred[i] == 4:
                correct += 1
                #use_correct +=1
            elif label[i] == 1 and pred[i] == 1:
                correct += 1
            elif label[i] == 1 and pred[i] == 3:
                correct += 1
                #use_correct +=1
            elif label[i] == 2 and pred[i] == 2:
                correct += 1
    acc = correct/total
    return acc

class MultiTaskModel:
    
    def __init__(self, regressor_model1, regressor_model2, classifier_model):
        self.regressor_1 = regressor_model1
        self.regressor_2 = regressor_model2
        self.classifier = classifier_model
        
    def fit(self, X1, X2, X3, y_reg_1, y_reg_2, y_cls):
        self.regressor_1.fit(X1, y_reg_1)
        self.regressor_2.fit(X2, y_reg_2)
        self.classifier.fit(X3, y_cls)
    
    def predict(self, X1, X2, X3):
        y_pred_reg_1 = self.regressor_1.predict(X1)
        y_pred_reg_2 = self.regressor_2.predict(X2)
        y_pred_cls = self.classifier.predict(X3)
        return y_pred_reg_1, y_pred_reg_2, y_pred_cls
        
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)        
wrapper1 = MultiOutputRegressor(LinearSVR( dual='auto'))
wrapper2 = MultiOutputRegressor(LinearSVR( dual='auto'))

models = {
    "RandomForest": MultiTaskModel(RandomForestRegressor(n_estimators=100, random_state=42), 
                                   RandomForestRegressor(n_estimators=100, random_state=42), RandomForestClassifier(n_estimators=100, random_state=42)),
    "KNN": MultiTaskModel(KNeighborsRegressor(), KNeighborsRegressor(), KNeighborsClassifier()), 
    "LR": MultiTaskModel(LinearRegression(), LinearRegression(), LogisticRegression()),
    "SVM": MultiTaskModel(wrapper1, wrapper2, SVC())
}

In [134]:
# Training
for name, model in models.items():
    for n in range(1):
        kfold = KFold(n_splits=num_fold, shuffle=True, random_state=n)
        all_reg1_test_y = []
        all_reg2_test_y = []
        all_cls_test_y = []
        all_reg1_test_p = []
        all_reg2_test_p = []
        all_cls_test_p = []
        train_idx_list = []
        val_idx_list = []
        #print(f"\nModel: {name}")
        for train_idx,test_idx in kfold.split(all_descriptors):
            
            train_cls_x, test_cls_x = all_descriptors[train_idx], all_descriptors[test_idx]
            train_cls_y, test_cls_y = target[train_idx],target[test_idx]
            train_idx_list.append(train_idx)
            val_idx_list.append(test_idx)
            
            train_reg1_x, test_reg1_x = DG_descs[train_idx], DG_descs[test_idx]
            train_reg1_y, test_reg1_y = target_DG[train_idx],target_DG[test_idx]
            
            train_reg2_x, test_reg2_x = RX_descs[train_idx], RX_descs[test_idx]
            train_reg2_y, test_reg2_y = target_RX[train_idx],target_RX[test_idx]
            
            model.fit(train_reg1_x, train_reg2_x, train_cls_x, train_reg1_y, train_reg2_y, train_cls_y)
            test_reg1_p, test_reg2_p, test_cls_p = model.predict(test_reg1_x, test_reg2_x, test_cls_x)
            all_reg1_test_y.append(test_reg1_y)
            all_reg2_test_y.append(test_reg2_y)
            all_cls_test_y.append(test_cls_y)
            all_reg1_test_p.append(test_reg1_p)
            all_reg2_test_p.append(test_reg2_p)
            all_cls_test_p.append(test_cls_p)
            
        all_reg1_test_y = np.concatenate(all_reg1_test_y)
        all_reg2_test_y = np.concatenate(all_reg2_test_y)
        all_cls_test_y = np.concatenate(all_cls_test_y)
        all_reg1_test_p = np.concatenate(all_reg1_test_p)
        all_reg2_test_p = np.concatenate(all_reg2_test_p)
        all_cls_test_p = np.concatenate(all_cls_test_p)
        train_idx = np.concatenate(train_idx_list)
        val_idx = np.concatenate(val_idx_list)
        
        cls_acc = evaluate_classification_accuracy(all_cls_test_y, all_cls_test_p, val_idx, ortho_sub_list, meta_sub_list)
        MAE_1 = mean_absolute_error(all_reg1_test_y, all_reg1_test_p)
        MAE_2 = mean_absolute_error(all_reg2_test_y, all_reg2_test_p)
        all_reg1_test_y = all_reg1_test_y.flatten()
        all_reg1_test_p = all_reg1_test_p.flatten()
        r2_1, _ = pearsonr(all_reg1_test_y, all_reg1_test_p)
        all_reg2_test_y = all_reg2_test_y.flatten()
        all_reg2_test_p = all_reg2_test_p.flatten()
        r2_2, _ = pearsonr(all_reg2_test_y, all_reg2_test_p)
        
        print ("{} {} site-selectivity accuraccy {:.4f} ".format(name, n+1, cls_acc))
       


RandomForest 1 site-selectivity accuraccy 0.8984 
KNN 1 site-selectivity accuraccy 0.7148 
LR 1 site-selectivity accuraccy 0.2188 
SVM 1 site-selectivity accuraccy 0.6562 
